In [ ]:
import json
import re

with open("train.json", "r") as f:
    data = json.load(f)

def clean_ing(ing):
    ing = ing.lower()
    ing = re.sub(r"\([^)]*\)", "", ing)   # убрать скобки
    ing = re.sub(r"\s+", " ", ing).strip()
    return ing

cleaned = []
for r in data:
    cleaned.append({
        "id": r["id"],
        "cuisine": r["cuisine"],
        "ingredients": [clean_ing(i) for i in r["ingredients"]]
    })

with open("recipes_clean.json", "w") as f:
    json.dump(cleaned, f, indent=2)

print("Готово: recipes_clean.json")

Готово: recipes_clean.json


In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

with open("recipes_clean.json", "r", encoding="utf-8") as f:
    recipes = json.load(f)

output = []

for i, r in enumerate(recipes):
    text = "Ingredients: " + ", ".join(r["ingredients"])
    emb = model.encode(text).tolist()

    output.append({
        "id": r["id"],
        "cuisine": r["cuisine"],
        "ingredients": r["ingredients"],
        "embedding": emb
    })

    if i % 500 == 0:
        print(f"Processed {i}/{len(recipes)}")

with open("recipes_with_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(output, f)

print("Готово: recipes_with_embeddings.json")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processed 0/39774
Processed 500/39774
Processed 1000/39774
Processed 1500/39774
Processed 2000/39774
Processed 2500/39774
Processed 3000/39774
Processed 3500/39774
Processed 4000/39774
Processed 4500/39774
Processed 5000/39774
Processed 5500/39774
Processed 6000/39774
Processed 6500/39774
Processed 7000/39774
Processed 7500/39774
Processed 8000/39774
Processed 8500/39774
Processed 9000/39774
Processed 9500/39774
Processed 10000/39774
Processed 10500/39774
Processed 11000/39774
Processed 11500/39774
Processed 12000/39774
Processed 12500/39774
Processed 13000/39774
Processed 13500/39774
Processed 14000/39774
Processed 14500/39774
Processed 15000/39774
Processed 15500/39774
Processed 16000/39774
Processed 16500/39774
Processed 17000/39774
Processed 17500/39774
Processed 18000/39774
Processed 18500/39774
Processed 19000/39774
Processed 19500/39774
Processed 20000/39774
Processed 20500/39774
Processed 21000/39774
Processed 21500/39774
Processed 22000/39774
Processed 22500/39774
Processed 23

In [ ]:
import json
import os
import math
from tqdm import tqdm

# === НАСТРОЙКИ ===
SOURCE = "recipes_with_embeddings.json"   # твой большой файл
CHUNK_SIZE_MB = 20                        # размер одного чанка
OUT_DIR = "chunks"                        # сюда будут записаны части
# ==================


# Создаём папку для выходных файлов
os.makedirs(OUT_DIR, exist_ok=True)

# 1. Загружаем большой JSON
print("📥 Loading original JSON...")
with open(SOURCE, "r") as f:
    data = json.load(f)

total = len(data)
print(f"✔ Loaded {total} recipes")


# 2. Прикидываем размер одного рецепта
# (примерно, но достаточно точно)
sample_bytes = len(json.dumps(data[0], ensure_ascii=False).encode("utf-8"))
sample_kb = sample_bytes / 1024
print(f"📏 Approx size of 1 recipe: {sample_kb:.2f} KB")

# 3. Считаем оптимальное количество рецептов на чанк
recipes_per_chunk = int((CHUNK_SIZE_MB * 1024) / sample_kb)
print(f"📦 Recipes per chunk ≈ {recipes_per_chunk}")

# 4. Считаем количество кусков
num_chunks = math.ceil(total / recipes_per_chunk)
print(f"🔪 Total chunks needed: {num_chunks}")

# 5. Реальное дробление
print("\n🔄 Splitting...")
for i in tqdm(range(num_chunks)):
    start = i * recipes_per_chunk
    end = start + recipes_per_chunk
    chunk = data[start:end]

    out_path = os.path.join(OUT_DIR, f"part{i+1}.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(chunk, f, ensure_ascii=False)

    size_mb = os.path.getsize(out_path) / (1024 * 1024)
    print(f"✔ Saved part{i+1}.json — {size_mb:.2f} MB")


print("\n🎉 DONE!")
print(f"➡ All chunks saved in folder: {OUT_DIR}/")
print("Upload the ENTIRE 'chunks' folder to your GitHub project root.")


📥 Loading original JSON...
✔ Loaded 39774 recipes
📏 Approx size of 1 recipe: 8.44 KB
📦 Recipes per chunk ≈ 2426
🔪 Total chunks needed: 17

🔄 Splitting...


  6%|▌         | 1/17 [00:03<00:49,  3.12s/it]

✔ Saved part1.json — 20.06 MB


 12%|█▏        | 2/17 [00:06<00:46,  3.13s/it]

✔ Saved part2.json — 20.06 MB


...

✔ Saved part15.json — 20.07 MB


 94%|█████████▍| 16/17 [00:36<00:02,  2.78s/it]

✔ Saved part16.json — 20.07 MB


100%|██████████| 17/17 [00:37<00:00,  2.19s/it]

✔ Saved part17.json — 7.93 MB

🎉 DONE!
➡ All chunks saved in folder: chunks/
Upload the ENTIRE 'chunks' folder to your GitHub project root.
